# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [35]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [36]:
cities = pd.read_csv("../output_data/cities.csv", encoding="utf-8")
cities

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Port Macquarie,0,AU,1593574378,42,-31.43,152.92,68.00,1.01
1,Mackenzie,40,CA,1593574379,77,55.30,-123.17,60.80,3.36
2,Loreto,100,MX,1593574381,52,22.27,-101.97,69.84,16.73
3,Albany,99,US,1593574201,92,42.60,-73.97,64.99,1.92
4,Jamestown,1,US,1593574383,82,42.10,-79.24,64.40,2.64
...,...,...,...,...,...,...,...,...,...
553,Riyadh,0,SA,1593575003,28,24.69,46.72,82.40,5.21
554,Ati,90,TD,1593575005,58,13.22,18.34,81.61,16.71
555,Natal,75,BR,1593575006,100,-5.79,-35.21,73.40,4.70
556,Tepechitlán,95,MX,1593575007,77,21.67,-103.33,65.52,4.47


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [37]:
humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Lat", "Lng"]]

In [38]:
fig_map = gmaps.figure()
heat_map = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig_map.add_layer(heat_map)
fig_map

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [39]:
pref_city_df = cities.loc[(cities["Cloudiness"] == 0) & cities["Max Temp"] >= 60 & (cities["Max Temp"] <= 80)].dropna()

pref_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Port Macquarie,0,AU,1593574378,42,-31.43,152.92,68.00,1.01
1,Mackenzie,40,CA,1593574379,77,55.30,-123.17,60.80,3.36
2,Loreto,100,MX,1593574381,52,22.27,-101.97,69.84,16.73
3,Albany,99,US,1593574201,92,42.60,-73.97,64.99,1.92
4,Jamestown,1,US,1593574383,82,42.10,-79.24,64.40,2.64
...,...,...,...,...,...,...,...,...,...
553,Riyadh,0,SA,1593575003,28,24.69,46.72,82.40,5.21
554,Ati,90,TD,1593575005,58,13.22,18.34,81.61,16.71
555,Natal,75,BR,1593575006,100,-5.79,-35.21,73.40,4.70
556,Tepechitlán,95,MX,1593575007,77,21.67,-103.33,65.52,4.47


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [40]:
# Create a hotel_df
hotel_df = pref_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Port Macquarie,AU,-31.43,152.92,
1,Mackenzie,CA,55.30,-123.17,
2,Loreto,MX,22.27,-101.97,
3,Albany,US,42.60,-73.97,
4,Jamestown,US,42.10,-79.24,
...,...,...,...,...,...
553,Riyadh,SA,24.69,46.72,
554,Ati,TD,13.22,18.34,
555,Natal,BR,-5.79,-35.21,
556,Tepechitlán,MX,21.67,-103.33,


In [41]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}
print("Pulling Hotels That Meet Standards")
print(f"-------------------------------")
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Result {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print(f"-------------------------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Pulling Hotels That Meet Standards
-------------------------------
Result 0: Port Macquarie.
Closest hotel in Port Macquarie is Sails Port Macquarie by Rydges.
-------------------------------
Result 1: Mackenzie.
Closest hotel in Mackenzie is Alexander Mackenzie Hotel.
-------------------------------
Result 2: Loreto.
Closest hotel in Loreto is Hotel San Miguel.
-------------------------------
Result 3: Albany.
Missing field/result... skipping.
-------------------------------
Result 4: Jamestown.
Closest hotel in Jamestown is Chautauqua Harbor Hotel.
-------------------------------
Result 5: Kununurra.
Closest hotel in Kununurra is Freshwater East Kimberley Apartments | Kununurra Accommodation.
-------------------------------
Result 6: Punta Arenas.
Closest hotel in Punta Arenas is Hotel Isla Rey Jorge.
-------------------------------
Result 7: Beernem.
Missing field/result... skipping.
-------------------------------
Result 8: Bathsheba.
Closest hotel in Bathsheba is ECO Lifestyle & L

Result 80: Monrovia.
Closest hotel in Monrovia is The Royal Grand Hotel.
-------------------------------
Result 81: Anadyr.
Closest hotel in Anadyr is Chukotka hotel.
-------------------------------
Result 82: Nanortalik.
Closest hotel in Nanortalik is Hotel Kap Farvel.
-------------------------------
Result 83: Molina.
Closest hotel in Molina is Hostal Radal.
-------------------------------
Result 84: Waddān.
Missing field/result... skipping.
-------------------------------
Result 85: Bengkulu.
Closest hotel in Bengkulu is Hotel Santika Bengkulu.
-------------------------------
Result 86: Cabo San Lucas.
Closest hotel in Cabo San Lucas is Sandos Finisterra Los Cabos.
-------------------------------
Result 87: Bethel.
Closest hotel in Bethel is Courtyard by Marriott Danbury.
-------------------------------
Result 88: Esperance.
Closest hotel in Esperance is The Jetty Resort.
-------------------------------
Result 89: Bambanglipuro.
Closest hotel in Bambanglipuro is Paddy D'sawah Guest 

Result 161: Katsuura.
Closest hotel in Katsuura is Kuroshio no Yado Rinkaisou, Minami-Boso Katsuura Beach.
-------------------------------
Result 162: Wasco.
Closest hotel in Wasco is Best Western Plus Wasco Inn & Suites.
-------------------------------
Result 163: Liverpool.
Closest hotel in Liverpool is DoubleTree by Hilton Hotel & Spa Liverpool.
-------------------------------
Result 164: Madang.
Closest hotel in Madang is Madang Lodge Hotel.
-------------------------------
Result 165: Athani.
Closest hotel in Athani is Pattan's Hotel Prathamesh Arcade.
-------------------------------
Result 166: Mehamn.
Closest hotel in Mehamn is Mehamn Arctic Hotel AS.
-------------------------------
Result 167: Komsomol'skoye.
Closest hotel in Komsomol'skoye is Gostinyy Dvor "Oazis".
-------------------------------
Result 168: Najrān.
Missing field/result... skipping.
-------------------------------
Result 170: Kirakira.
Missing field/result... skipping.
-------------------------------
Result 171

Result 239: Yantal'.
Missing field/result... skipping.
-------------------------------
Result 240: Antsohihy.
Closest hotel in Antsohihy is Hôtel le Paradisier.
-------------------------------
Result 241: Pimentel.
Closest hotel in Pimentel is Hotel Puerto del Sol.
-------------------------------
Result 242: Araouane.
Missing field/result... skipping.
-------------------------------
Result 243: Hofn.
Closest hotel in Hofn is Hótel Höfn.
-------------------------------
Result 244: Kalianget.
Missing field/result... skipping.
-------------------------------
Result 245: Sorong.
Closest hotel in Sorong is Swiss-Belhotel Sorong.
-------------------------------
Result 246: Iqaluit.
Closest hotel in Iqaluit is Frobisher Inn.
-------------------------------
Result 247: Carutapera.
Closest hotel in Carutapera is HOTEL R. SOARES.
-------------------------------
Result 248: Road Town.
Closest hotel in Road Town is Sea View Hotel.
-------------------------------
Result 249: San Nicolas.
Closest ho

Result 319: Konya Province.
Missing field/result... skipping.
-------------------------------
Result 320: Inhambane.
Closest hotel in Inhambane is Hotel Casa Do Capitão.
-------------------------------
Result 321: Zhigansk.
Closest hotel in Zhigansk is Gostinichnyy Kompleks Aylgy.
-------------------------------
Result 322: Talcahuano.
Closest hotel in Talcahuano is Hotel Düsseldorf.
-------------------------------
Result 323: Deputatsky.
Missing field/result... skipping.
-------------------------------
Result 324: Carrboro.
Closest hotel in Carrboro is AC Hotel by Marriott Chapel Hill Downtown.
-------------------------------
Result 325: Acaraú.
Closest hotel in Acaraú is Hotel Acaraú Riviera.
-------------------------------
Result 326: Touros.
Closest hotel in Touros is Hotel Vila Gale Touros.
-------------------------------
Result 327: Oeiras.
Closest hotel in Oeiras is Hotel SESC Oeiras.
-------------------------------
Result 328: Alta Floresta.
Closest hotel in Alta Floresta is Ho

Result 401: Umluj.
Closest hotel in Umluj is Juman Hotel.
-------------------------------
Result 402: Winnemucca.
Closest hotel in Winnemucca is Candlewood Suites Winnemucca.
-------------------------------
Result 403: Skegness.
Closest hotel in Skegness is The Royal Hotel.
-------------------------------
Result 404: Xining.
Closest hotel in Xining is Sofitel Xining.
-------------------------------
Result 405: Dingle.
Missing field/result... skipping.
-------------------------------
Result 406: Bida.
Closest hotel in Bida is Desert Prince Hotel.
-------------------------------
Result 407: Guánica.
Closest hotel in Guánica is Copamarina Beach Resort & Spa.
-------------------------------
Result 408: Margate.
Closest hotel in Margate is Sands Hotel Margate.
-------------------------------
Result 409: Manggar.
Closest hotel in Manggar is OYO 515 Oasis Hotel.
-------------------------------
Result 410: Shubarkuduk.
Missing field/result... skipping.
-------------------------------
Result 41

Result 480: Córdoba.
Closest hotel in Córdoba is Onas Hostel & Suites.
-------------------------------
Result 481: Prome.
Closest hotel in Prome is Lucky Dragon Hotel.
-------------------------------
Result 482: Muros.
Closest hotel in Muros is Hotel La Muradana.
-------------------------------
Result 483: Igarka.
Closest hotel in Igarka is Zapolyar'ye, Gostinitsa.
-------------------------------
Result 484: Hobyo.
Closest hotel in Hobyo is Osmani Hotel.
-------------------------------
Result 485: Burley.
Closest hotel in Burley is Hampton Inn Burley.
-------------------------------
Result 486: Berlevåg.
Closest hotel in Berlevåg is Berlevåg camping & Appartement AS.
-------------------------------
Result 487: Verkhoshizhem'ye.
Closest hotel in Verkhoshizhem'ye is Forest.
-------------------------------
Result 488: Antigonish.
Closest hotel in Antigonish is Antigonish Victorian Inn.
-------------------------------
Result 489: Mahébourg.
Closest hotel in Mahébourg is Shandrani Beachcomb

In [43]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [44]:
# Add marker layer ontop of heat map
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig_map.add_layer(markers)

# Display figure
fig_map

Figure(layout=FigureLayout(height='420px'))